In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [3]:
#geopy installation

!pip install geopy
import geopy 

from geopy.geocoders import GoogleV3
from geopy.geocoders import Nominatim
import time
from pprint import pprint
geolocatorGoogle = GoogleV3(api_key=" ")
geolocatorNom = Nominatim(user_agent="cphDorms")

In [4]:
#Installing shortuuid to make dorms identifiable (Some Dorm Names Suck!)

!pip install shortuuid

import shortuuid    
print(shortuuid.ShortUUID().random(length=6)) #Example Code 

39Kwc2


In [6]:
url = "https://www.kollegierneskontor.dk/default.aspx?func=kkikportal.kollegiumlist&mid=40&topmenuid=34&lang=GB"
results = requests.get(url)

In [7]:
soup = BeautifulSoup(results.text, "html.parser")


In [17]:
#Longitude and latitude

def coordinatesGoogle(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocatorGoogle.geocode(address)
             return {"latitude":location.latitude, "longitude":location.longitude}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":"NaN", "longitude":"NaN"}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location

#print(f"{latitude}, {longitude}")
# print all returned data
#pprint(location)

def coordinatesNomantim(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocatorNom.geocode(address).raw
             return {"latitude":location['lat'], "longitude":location['lon']}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":"NaN", "longitude":"NaN"}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location


print(coordinatesNomantim("Dalslandsgade 8, Copenhagen"))

8, Dalslandsgade, Christianshavn, København, Københavns Kommune, Region Hovedstaden, 2300, Danmark
{'latitude': '55.667422', 'longitude': '12.60431'}


In [8]:

kkik = []

kollegium = soup.find_all('div', class_='kollegium row')


for container in kollegium:
    print("Processing Dorm")
    dorm_name = container.h3.a.text
    dorm_address = container.div.find('div', class_="address").text.strip().replace("\r\n               ", "").replace("\xa0", "")
    dorm_picture_link = container.find('img', class_="kollegiumlistimage img")['src']
    print(dorm_picture_link)

    body = container.find('div', class_="body col-xs-12")
    rows = body.find_all("div", class_= "row")
    rows.pop(0)
    rows.pop(0)
    rows.pop(0)

    rooms_export = []

    for room in rows:
        room_details = room.find_all("div")

        roomtype = room_details[0].get_text().strip()
        size = room_details[1].get_text().strip()
        price = room_details[3].get_text().strip()

        rooms_export.append({
            "roomType":roomtype,
            "size":size,
            "price":price
        })

    kkik.append({
        "name":dorm_name,
        "address":dorm_address,
        "dormPictureLink":"https://www.kollegierneskontor.dk/"+dorm_picture_link,
        "coordinates":coordinatesNomantim(dorm_address),
        "rooms":rooms_export,
        "UUID":shortuuid.ShortUUID().random(length=6)
    })








SyntaxError: invalid syntax (<ipython-input-8-6a4dadb4d483>, line 38)

In [28]:
import json

with open('json_kkik.json', 'w') as f:
    json.dump(kkik, f)


3Y9pHV
